In [15]:
import numpy as np


np.random.seed(1)
A_prev = np.random.randn(2, 5, 7, 4)
W = np.random.randn(3, 3, 4, 8)
b = np.random.randn(1, 1, 1, 8)
hparameters = {"pad" : 1,
               "stride": 1
               }





In [16]:
def conv_single_step(a_slice_prev, W, b):
    """
    Apply one filter defined by parameters W on a single slice (a_slice_prev) of the output activation
    of the previous layer.

    Arguments:
    a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
    W -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
    b -- Bias parameters contained in a window - matrix of shape (1, 1, 1)

    Returns:
    Z -- a scalar value, the result of convolving the sliding window (W, b) on a slice x of the input data
    """
    s = a_slice_prev * W
    Z = np.sum(s)
    Z = Z + float(b)

    return Z

In [17]:
def conv_forward(A_prev, W, b, hparameters):
    """
    Implements the forward propagation for a convolution function

    Arguments:
    A_prev -- output activations of the previous layer,
        numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
    b -- Biases, numpy array of shape (1, 1, 1, n_C)
    hparameters -- python dictionary containing "stride" and "pad"

    Returns:
    Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache of values needed for the conv_backward() function
    """

    (m ,n_H_prev, n_W_prev, n_C_prev)  = A_prev.shape

    (f ,f ,n_C_prev,n_c) = W.shape

    stride = hparameters["stride"]
    pad = hparameters["pad"]

    # Compute the dimensions of the CONV output volume using the formula given above.

    n_H = int(((n_H_prev - f + (2*pad)) / stride)) +1
    n_W = int(((n_W_prev - f + (2*pad)) / stride)) +1
    # Initialize the output volume Z with zeros

    Z = np.zeros((m,n_H ,n_W,n_c))
    # Create A_prev_pad by padding A_prev
    A_prev_pad = np.pad(A_prev ,((0,0), (pad,pad), (pad,pad) ,(0,0)) ,mode ="constant" ,constant_values=(0,0))
    # loop over the batch of training examples

    for i in range(m): # Select ith training example's padded activation

        a_prev_pad = A_prev_pad[i]
        for h in range(n_H): # loop over vertical axis of the output volume
            # Find the vertical start and end of the current
            vert_start = h * stride
            vert_end = vert_start + f


            for w in range(n_W):  # loop over horizontal axis of the output volume
                    # Find the horizontal start and end of the current
                    horiz_start = w * stride
                    horiz_end = horiz_start + f

                    # loop over channels (= #filters) of the output volume
                    for c in range(n_c):
                        # Use the corners to define the (3D) slice of a_prev_pad
                        a_slice_prev = a_prev_pad[vert_start:vert_end ,horiz_start:horiz_end ,:]
                        # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. (≈3 line)
                        weights = W[:,: ,: ,c]
                        biases = b[:,:, :, c]
                        Z[i ,h ,w, c] = conv_single_step(a_slice_prev, weights, biases)

    # Save information in "cache" for the backprop
    cache = (A_prev, W, b, hparameters)

    return Z, cache

In [18]:
 Z, cache_conv = conv_forward(A_prev, W, b, hparameters)


<ipython-input-16-122cd41bb878>:18: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Z = Z + float(b)


In [25]:
Z, cache_conv

(array([[[[ -2.65112363,  -0.37849177,  -1.97054929,  -1.96235299,
            -1.72259872,   0.4676693 ,  -6.43434016,   1.10764994],
          [  4.67692928,   4.29865415,  -1.3608031 ,   0.80532859,
            -2.88480108,   8.95280034,   5.32627807,  -1.82635258],
          [ -2.05881174,   3.40859795,   0.3502282 ,   0.68303626,
            -1.88328065,  -1.87480174,   5.8008721 ,   0.0700918 ],
          [ -3.50141791,   2.704286  ,   0.28341346,   4.15637411,
            -0.46575834,  -0.43668824,  -5.56866106,   1.72288033]],
 
         [[ -2.32126108,   0.91040602,   2.31852532,   0.98842271,
             3.31716611,   4.05638832,  -2.48135123,   0.95872443],
          [  6.03978907,  -6.96477888,  -1.20799344,   2.68913374,
            -4.35744033,  10.59355329,   3.20856901,  13.98735978],
          [ -3.01280755,  -2.90226517,  -8.34171936,  -5.26220853,
             5.6630696 ,   1.08704033,   2.20430705, -10.73218294],
          [ -6.24198266,  -0.53158832,  -3.29654954,

In [22]:
def pool_forward(A_prev, hparameters,f, mode = "max"):
    """
    Implements the forward pass of the pooling layer

    Arguments:
    A_prev -- Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- python dictionary containing "f" and "stride"
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")

    Returns:
    A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache used in the backward pass of the pooling layer, contains the input and hparameters
    """

    # Retrieve dimensions from the input shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

    # Retrieve hyperparameters from "hparameters"

    stride = hparameters["stride"]

    # Define the dimensions of the output
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev

    # Initialize output matrix A
    A = np.zeros((m, n_H, n_W, n_C))


    # loop over the training examples
    for i in range(m):
        # loop on the vertical axis of the output volume
        for h in range(n_H):
            vert_start = h * stride
            vert_end = vert_start + f

            # loop on the horizontal axis of the output volume
            for w in range(n_W):
                horiz_start = w * stride
                horiz_end = horiz_start + f


               # loop over the channels of the output volume
                for c in range(n_C_prev):
                    # Use the corners to define the current slice on the ith training example of A_prev, channel c.
                    a_slice_prev = A_prev[ i, vert_start:vert_end ,horiz_start:horiz_end ,c]

                    # Compute the pooling operation on the slice.
                    # Use an if statement to differentiate the modes.
                    # Use np.max and np.mean.
                    # if mode == "max":
                    if mode == "max":
                        A[i, h, w ,c] = np.max(a_slice_prev)

                    elif mode == "average":
                        A[i , h ,w ,c] = np.mean(a_slice_prev)


    # Store the input and hparameters in "cache" for pool_backward()
    cache = (A_prev, hparameters)

    # Making sure your output shape is correct
    assert A.shape == (m, n_H, n_W, n_C)

    return A, cache

In [23]:
A, cache = pool_forward(Z, hparameters,3, mode = "max")

In [24]:
A, cache

(array([[[[ 6.03978907,  8.07171329,  2.31852532,  3.36286738,
            5.6630696 , 10.59355329, 10.99288867, 13.98735978]]],
 
 
        [[[11.54630784,  6.97041391,  8.33521755,  7.62184275,
            5.40584348,  7.32417919,  4.1356109 ,  7.91314719]]]]),
 (array([[[[ -2.65112363,  -0.37849177,  -1.97054929,  -1.96235299,
             -1.72259872,   0.4676693 ,  -6.43434016,   1.10764994],
           [  4.67692928,   4.29865415,  -1.3608031 ,   0.80532859,
             -2.88480108,   8.95280034,   5.32627807,  -1.82635258],
           [ -2.05881174,   3.40859795,   0.3502282 ,   0.68303626,
             -1.88328065,  -1.87480174,   5.8008721 ,   0.0700918 ],
           [ -3.50141791,   2.704286  ,   0.28341346,   4.15637411,
             -0.46575834,  -0.43668824,  -5.56866106,   1.72288033]],
  
          [[ -2.32126108,   0.91040602,   2.31852532,   0.98842271,
              3.31716611,   4.05638832,  -2.48135123,   0.95872443],
           [  6.03978907,  -6.96477888,  -1.207